# HOMEWORK 05

**Submitted by: Diego Sol**

CSCE 636-600: Deep Learning

Professor: Dr. Anxiao Jiang

## Task 1

**Check out the Jupyter notebook for Chapter 8 at https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/chapter08_intro-to-dl-for-computer-vision.ipynb. It uses the convolutional base of VGG16 for an image classification task, and also tries fine tuning. Your task: use another existing trained neural network (which is different from VGG16, such as ResNet) for the same task, and also try fine tuning. Submit your complete code, draw figures on the training/validation performance, and show the testing performance.**

Upload `kaggle.json` file

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"diegosol","key":"f60bcb6235ead4cd779aa85e779457cc"}'}

Create directory and download data

In [3]:
# create a Kaggle folder
!mkdir ~/.kaggle
# copy the key file to the folder
!cp kaggle.json ~/.kaggle/
# make file readable only by the user
!chmod 600 ~/.kaggle/kaggle.json

# download the data
!kaggle competitions download -c dogs-vs-cats

# unzip data
!unzip -qq dogs-vs-cats.zip
!unzip -qq train.zip

 98% 798M/812M [00:02<00:00, 292MB/s]
100% 812M/812M [00:02<00:00, 306MB/s]


Copy images to training, validation, and test directories

In [4]:
import os, shutil, pathlib

# directory of original uncompressed data
original_dir = pathlib.Path("train")
# directory where we will store the smaller dataset
new_base_dir = pathlib.Path("cats_vs_dogs_small")

# function that copies indexed images of cats and dogs to subdirectories
def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)

# training subset
make_subset("train", start_index=0, end_index=1000)
# validation subset
make_subset("validation", start_index=1000, end_index=1500)
# test subset
make_subset("test", start_index=1500, end_index=2500)

Read images

In [5]:
from tensorflow.keras.utils import image_dataset_from_directory

# training data
train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(180, 180),
    batch_size=32)

# validation data
validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size=(180, 180),
    batch_size=32)

# test data
test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(180, 180),
    batch_size=32)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


Instantiating the ResNet50V2 convolutional base

In [6]:
from tensorflow import keras

conv_base = keras.applications.ResNet50V2(
    include_top=False,
    weights="imagenet",
    input_shape=(180, 180, 3),
    pooling="avg",
)

94683136/94668760 [==============================] - 0s 0us/step
